<a href="https://colab.research.google.com/github/plotia/Computational-Methods-for-Biological-Modelling-and-Simulation/blob/main/HW6_02_712_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Expectation-Maximization for Metagenomic Community Analysis

In this problem, we will explore the use of expectation-maximization (EM) for a metagenomic community analysis. Our objective is to estimate the frequencies of different bacterial strains based on the frequencies of alleles at a set of variant sites that differ between strains. This problem is challenging because sequencing machines generally provide only small pieces of sequence that are unlikely to contain more than one variant site.

We want to determine how common each strain is, but the value cannot be derived easily because we do not know which strain contains a particular variant allele. For example, assume we have three variant sites and three bacterial strains A, B, and C: strain A has minor alleles at sites 1 and 2, strain B has minor alleles at sites 2 and 3, and strain C has minor alleles at sites 1 and 3. If we observe a high frequency of the minor allele at site 2, we do not know how much of that frequency should be attributed to strain A and how much to strain B. Our goal is to take an observed minor allele frequency at each site and infer the frequencies of the strains.

To formalize this problem, assume we have `n` variant sites and `m` bacterial strains. Define the variable `eij` to be 1 if strain `i` has the minor allele at site `j`, and 0 otherwise. Assume that the `eij` values are known. We also assume that our input includes a total count `C` of sequence reads for each site `i`, and a count `xi` representing the number of those reads containing the minor allele at that site. Our goal is to infer a set of frequencies λ = [f1, …, fm], corresponding to the frequencies of the `m` strains.

### a. Likelihood Function
Define a likelihood function for this problem expressing the probability of the allele observations x = [x1, …, xn] given the model λ = [f1, …, fm] and the read counts `C`.


Ans.
Lets assume i $∈$ [1,..,n] & j $∈$ [1,..,m]

For a single site $ i $, the probability of finding an allele at site i,  $ p_i $ is computed as:
$
p_i = \sum_{j=1}^m f_j E_{j,i}
$

Now, to calculate the likelihood $ \Pr(x_i | \text{C}) $:
$
\Pr(x_i | \text{C}, \vec\lambda) = \binom{C}{x_i} \left( \sum_{j=1}^m f_j E_{j,i} \right)^{x_i} \left( 1 - \sum_{j=1}^m f_j E_{j,i} \right)^{C - x_i}
$

The overall likelyhood is:

$
L(\lambda) = \prod_{i=1}^n \binom{C}{x_i} \left( \sum_{j=1}^m f_j E_{j,i} \right)^{x_i} \left( 1 - \sum_{j=1}^m f_j E_{j,i} \right)^{C - x_i}
$

Where:
- $ \binom{C_i}{x_i} $: Binomial coefficient for total reads $( C_i )$ and minor allele reads $( x_i )$.
- $ \sum_{j=1}^m \lambda_j e_{ij} $: The probability of observing the minor allele at site $ i $, which depends on the contributions of all strains.


### b. M-Step
To find the maximum-likelihood frequencies using EM, we introduce latent variables `yij`, defined as the number of allele counts at site `i` that really came from strain `j`. Using this definition, define the M-step of the algorithm, i.e., estimate the frequencies `f1, …, fm` of observing a sequence from each strain given the `xj`'s and `yij`'s. Assume that there are no sequencing errors.


Ans. **Maximization step**

Lets create a matrix $Y$ filled with latent variables $y_{ij}$.

Hence $Y$ is a $n$ x $m$ matrix as i  ∈  [1,..,n] & j  ∈  [1,..,m]

$y_{ij}$ determines the number of allele counts from a variant site $i$ in strain $j$

Hence,

$y_{i,j} = x_i \frac{f_j E_{j,i}}{\sum_{j=1}^m f_j E_{j,i}}$

### c. E-Step
To specify the E-step, each `fi` should be expressed as a linear function of the `yij` values. Define the expected value of each `yij` given x and λ to specify the E-step.


Ans. **Expectation step**

Here, we estimate the new frequencies using data provided by the latent variables and total count of sequences studied

Frequency update:

 $f_j^{(t+1)} = \frac{\sum_{i=1}^n y_{i,j}}{n C}$

### d. Pseudocode for EM Inference Method
Write pseudocode specifying a complete inference method for λ given x. Initialize by assuming equal frequencies for each strain, and assume a user-specified number of rounds `r` of EM.


Ans.
### Pseudocode for Expectation-Maximization Algorithm:

   - Define an array $\lambda$ with equal frequencies $ f_j = \frac{1}{m} $ for $ j \in [1, m] $
   - Define m strains and n allele sites within each strains with readings taken for C sequences along with an array x containing data count for presence of each allele
   - Define an $ m \times n $ matrix $ E $ containing presence or absence of minor allele data at every location of that allele within that strain. Rows determine strains and columns determine minor allele sites
   - Define a latent variable $y_{i,j} = x_i \frac{f_j E_{j,i}}{\sum_{j=1}^m f_j E_{j,i}}$ where $x_i \in x$ and $i \in [1,n]$. I denotes the site of the allele and j demotes the strain type
   - Define an $ n \times m $ matrix $ Y $ with latent variables $y_{ij}$
   - Determine new $ f_j$ using the formula, $f_j^{(t+1)} = \frac{\sum_{i=1}^n y_{i,j}}{n C}$ to update the frequencies
   - Based on new frequencies, repeat steps 4 to 6 for $r$ times
   - At the $r^{th}$ iteration, determine $ \lambda $

### e. Code Implementation
Write code implementing your EM inference method. The code should take as input:
- The dimensions `m` and `n`
- An `m × n` matrix `E` specifying the splice forms
- The total read depth `c`
- The vector of minor allele counts `[x1, …, xn]`
- The number of EM rounds `r`

The code should return the estimated strain frequencies `[f1, …, fm]`.


In [7]:
import numpy as np

def exp_max(m, n, E, C, x, r):
    # Initialize λ with equal frequencies
    λ = np.ones(m) / m
    y = np.zeros((n, m))

    print(f'Initialized frequencies to {λ}')

    def p(i):
      p_i = []
      for j in range(m):
        p_i.append(λ[j] * E[j][i])
      pi = sum(p_i)
      return pi

    def latent_variable(i,j):
      y_ij = x[i] * ((λ[j] * E[j][i])/(p(i)))
      return y_ij

    def exp(j):
      yi = []
      for i in range(n):
        yi.append(y[i][j])
      sum_yi = np.sum(yi)
      fj = sum_yi/(n*C)
      return fj

    for _ in range(r):

      for i in range(n):
        for j in range(m):
          y[i][j] = latent_variable(i,j)

      for j in range(m):
        λ[j] = exp(j)

    print(f'Final estimated strain frequencies are {λ}')

### f. Example Data Set
Provide your inferred frequencies for the following dataset:

- m = 3, n = 4
- Matrix `E`:
$$
\begin{bmatrix}
1 & 0 & 1 & 1 \\
1 & 1 & 0 & 0\\
0 & 1 & 1 & 0
\end{bmatrix}
$$
  
- Total read depth `C` = 50
- Minor allele counts: `[24, 16, 32, 19]`
- Number of EM rounds `r` = 100


In [8]:
# Test data
m, n = 3, 4
E = np.array([
    [1, 0, 1, 1],
    [1, 1, 0, 0],
    [0, 1, 1, 0]
])
C = 50
x = np.array([24, 16, 32, 19])
r = 100


In [8]:
exp_max(m,n,E,C,x,r)

Initialized frequencies to [0.33333333 0.33333333 0.33333333]
Final estimated strain frequencies are [0.33025005 0.00945263 0.11529732]
